In [1]:
!pip uninstall pyarrow -q
!pip install pyarrow -q
!pip install transformers -q
!pip install datasets -q
!pip install evaluate -q

Proceed (Y/n)? Y
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 7.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.10.1 requires pyarrow<18.0.0a0,>=14.0.0, but you have pyarrow 18.0.0 which is incompatible.
pylibcudf-cu12 24.10.1 requires pyarrow<18.0.0a0,>=14.0.0, but you have pyarrow 18.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This be

In [2]:
from transformers import pipeline, AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AutoModelForCausalLM, AutoModelForMaskedLM
from transformers import BertModel, BertConfig, BertTokenizer
from transformers import AdamW
from transformers import get_scheduler
from transformers import DataCollatorWithPadding
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import evaluate

import wandb
wandb.init(mode='disabled')

import torch
from torch.utils.data import DataLoader

import numpy as np

from tqdm.auto import tqdm
import html
import requests

In [5]:
checkpoint = "distilbert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [3]:
text = "This is a great [MASK]."

In [6]:
model_inputs = tokenizer(text, return_tensors='pt')
outputs = model(**model_inputs).logits
mask_token_index = torch.where(model_inputs["input_ids"] == tokenizer.mask_token_id)[1]
top_5_tokens = torch.topk(outputs[0, mask_token_index, :], 5, dim=1).indices[0].tolist()
tokenizer.decode(top_5_tokens)

'deal success adventure idea feat'

In [7]:
imdb_dataset = load_dataset("imdb")

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
imdb_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [9]:
imdb_dataset["train"][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [10]:
sample = imdb_dataset["unsupervised"].shuffle().select(range(3))

In [11]:
for s in sample:
  print(s['text'])
  print(s['label'])

I bought this movie. I had never seen it, but I figured since it had the "Return of the King" name it would be decent. I was horribly mistaken. I have tried to watch this movie about 3 times. Every single time I have fallen asleep only to be woken up by terrible music. IF you must see this film, rent it. Do not buy it. Save yourself from the torment of actually owning this movie.<br /><br />Why is it so bad?<br /><br />1) Terrible music sung by a guy with a terrible voice.<br /><br />2) Really doesn't follow what happened in the book (I know that sometimes movies cant follow a book exactly, but this movie doesn't even attempt to do so). 3) The music was terrible. 4) Very bad animation. Speed Racer has better animation.<br /><br />5) Bad music. 6) Did I mention the horrendous dialogue? Sam has around 15 minutes straight of dialogue that makes almost no sense. 7) I really hope I mentioned the horrific music.<br /><br />Those are only 7 of the many reasons I have for not liking this movie